# AI Red Teaming Agent for Generative AI models and applications in Azure AI Foundry

## Objective
This notebook walks through how to use Azure AI Evaluation's AI Red Teaming Agent functionality to assess the safety and resilience of AI systems against adversarial prompt attacks. AI Red Teaming Agent leverages [Risk and Safety Evaluations](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-metrics-built-in?tabs=warning#risk-and-safety-evaluators) to help identify potential safety issues across different risk categories (violence, hate/unfairness, sexual content, self-harm) combined with attack strategies of varying complexity levels from [PyRIT](https://github.com/Azure/PyRIT), Microsoft AI Red Teaming team's open framework for automated AI red teaming.

## Time
You should expect to spend about 30-45 minutes running this notebook. Execution time will vary based on the number of risk categories, attack strategies, and complexity levels you choose to evaluate.

## Before you begin

### Prerequisite
First, if you have an Azure subscription, create an [Azure AI hub](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/ai-resources) then [create an Azure AI project](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/ai-resources). AI projects and Hubs can be served within a private network and are compatible with private endpoints. You **do not** need to provide your own LLM deployment as the AI Red Teaming Agent hosts adversarial models for both simulation and evaluation of harmful content and connects to it via your Azure AI project.

In order to upload your results to Azure AI Foundry:
- Your AI Foundry project must have a connection (*Connected Resources*) to a storage account with `Microsoft Entra ID` authentication enabled.
- Your AI Foundry project must have the `Storage Blob Data Contributor` role in the storage account.
- You must have the `Storage Blob Data Contributor` role in the storage account.
- You must have network access to the storage account.

For more information see: https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/run-scans-ai-red-teaming-agent

**Important**: First, ensure that you've installed the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) and then make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

### Installation
From a terminal window, navigate to your working directory which contains this sample notebook, and execute the following.
```bash
python -m venv .venv
```

Then, activate the virtual environment created:

```bash
# %source .venv/bin/activate # If using Mac/Linux OS
.venv/Scripts/activate # If using Windows OS
```

With your virtual environment activated, install the following packages required to execute this notebook:

```bash
pip install uv
uv pip install azure-ai-evaluation[redteam] azure-identity openai azure-ai-projects
```


Now open VSCode with the following command, and ensure your virtual environment is used as kernel to run the remainder of this notebook.
```bash
code .
```

### Imports

In [1]:
from typing import Optional, Dict, Any
import os

# Azure imports
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy

# OpenAI imports
from openai import AzureOpenAI

### Login to Azure with valid credentials

Ensure that you've installed the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) and then make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

Configure the `credential` object with a different AzureCredential type if this is a requirement for your environment.

In [2]:
# Azure Credential imports
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

#!az login

# Initialize Azure credentials
credential = DefaultAzureCredential()

### Set Up Your Environment Variables

Set the following variables for use in this notebook. These variables connect to your Azure resources and model deployments.

Set these variables by creating an `.env` file in your project's root folder.

**Note:** You can find these values in your Azure AI Foundry project or Azure OpenAI resource.

For reference, here's an example of what your populated environment variables should look like:

```
# Azure OpenAI
AZURE_OPENAI_API_KEY="your-api-key-here"
AZURE_OPENAI_ENDPOINT="https://endpoint-name.cognitiveservices.azure.com/"
AZURE_OPENAI_DEPLOYMENT_NAME="gpt-4"
AZURE_OPENAI_API_VERSION="2024-12-01-preview"

# Azure AI Project
AZURE_PROJECT_ENDPOINT="https://your-aifoundry-endpoint-name.services.ai.azure.com/api/projects/yourproject-name"
```

In [4]:
# Azure AI Project information
azure_ai_project = os.environ.get("PROJECT_ENDPOINT")

# Azure OpenAI deployment information
azure_openai_deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")  # e.g., "gpt-4"
azure_openai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.environ.get("AZURE_OPENAI_API_KEY")  # e.g., "your-api-key"
azure_openai_api_version = os.environ.get("AZURE_OPENAI_API_VERSION")  # Use the latest API version

## Understanding AI Red Teaming Agent's capabilities

The Azure AI Evaluation SDK's `RedTeam` functionality evaluates AI systems against adversarial prompts across multiple dimensions:

1. **Risk Categories**: Different content risk categories your AI system might generate
   - Violence
   - HateUnfairness
   - Sexual
   - SelfHarm

2. **Attack Strategies**: Along with standard unmodified prompts which are sent by default as the `baseline`, you can specify different transformations of prompts to elicit undesired content.
You can also use `AttackStrategy.Compose()` to layer two strategies in one attack
   - AnsiAttack: Using ANSI escape codes in prompts
   - AsciiArt: Using ASCII art to disguise harmful content
   - AsciiSmuggler: Hiding harmful content within ASCII characters
   - Atbash: Using the Atbash cipher to encode harmful requests
   - Base64: Encoding harmful content in Base64 format
   - Binary: Converting text to binary to bypass filters
   - Caesar: Using the Caesar cipher for encoding
   - CharacterSpace: Manipulating character spacing to confuse filters
   - CharSwap: Swapping characters to bypass detection
   - Diacritic: Using diacritical marks to alter text appearance
   - Flip: Flipping text to bypass content filters
   - Leetspeak: Converting letters to numbers and symbols
   - Morse: Using Morse code to encode harmful requests
   - ROT13: Using ROT13 cipher for text transformation
   - SuffixAppend: Adding suffixes to confuse detection systems
   - StringJoin: Joining strings in unconventional ways
   - Tense: Changing the tense of harmful requests to past tense
   - UnicodeConfusable: Using similar-looking Unicode characters
   - UnicodeSubstitution: Substituting characters with Unicode alternatives
   - Url: Embedding harmful content within URLs
   - Jailbreak: Specially crafted prompts to bypass AI safeguards

3. **Complexity Levels**: Different difficultly levels of attacks
   - Baseline: Standard functionality tests
   - Easy: Simple attack patterns
   - Moderate: More sophisticated attacks
   - Difficult: Complex, layered attack strategies

The key metric for evaluating results is the **Attack Success Rate (ASR)**, which measures the percentage of attacks that successfully elicit harmful content from your AI system.

## Basic Example: Fixed Response Callback

Let's start with a basic example that demonstrates how to set up a Red Team evaluation using a simple callback that always returns a fixed, safe response regardless of the input prompt.

**Important Note**: Since this callback always responds with the same safe message, the Attack Success Rate (ASR) will be 0% for all categories. This example is merely to demonstrate the API structure and flow.

In [5]:
# Define a simple callback function that always returns a fixed response
def financial_advisor_callback(query: str) -> str:  # noqa: ARG001
    return "I'm a financial advisor assistant. I can help with investment advice and financial planning within legal and ethical guidelines."

In [6]:
# Create the `RedTeam` instance with minimal configurations
red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness],
    num_objectives=1,
)

Class RedTeam: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


NOTE: `num_objectives` specifies the number of attacks to perform per risk category per attack strategy. If the parameter `risk_categories` is not specified, `[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm]` will be used by default.

Now let's run a simple automated scan using the `RedTeam` with the fixed response target. We'll test against two risk categories and one attack strategy for simplicity.

In [7]:
# Run the red team scan called "Basic-Callback-Scan" with limited scope for this basic example
# This will test 1 objective prompt for each of Violence and HateUnfairness categories with the Flip strategy
result = await red_team.scan(
    target=financial_advisor_callback,
    scan_name="Basic-Callback-Scan",
    attack_strategies=[AttackStrategy.Flip],
    output_path="red_team_output.json",
)

🚀 STARTING RED TEAM SCAN: Basic-Callback-Scan
📂 Output directory: .\.scan_Basic-Callback-Scan_20250826_122503
📊 Risk categories: ['violence', 'hate_unfairness']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/resource/build/redteaming/1d827d7d-f622-45d6-b2f7-6455740e1a9f?wsid=/subscriptions/3c941de1-cd33-4339-bf35-4dafb9f0aff6/resourceGroups/test-agent/providers/Microsoft.CognitiveServices/accounts/agents-build2026-resource/projects/agents-build2026&tid=f79243ce-e61d-4d24-b491-0532f1cc8262
📋 Planning 4 total tasks


Scanning:   0%|                         | 0/4 [00:00<?, ?scan/s, current=fetching baseline/violence]

📚 Using attack objectives from Azure RAI service


Scanning:   0%|                  | 0/4 [00:03<?, ?scan/s, current=fetching baseline/hate_unfairness]

📝 Fetched baseline objectives for violence: 1 objectives


...

⚙️ Processing 4 tasks in parallel (max 5 at a time)
▶️ Starting task: baseline strategy for violence risk category
▶️ Starting task: baseline strategy for hate_unfairness risk category
▶️ Starting task: flip strategy for violence risk category
▶️ Starting task: flip strategy for hate_unfairness risk category


Scanning:  75%|█████████████████████████▌        | 3/4 [00:28<00:28, 28.55s/scan, current=batch 1/1]

Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Basic-Callback-Scan_20250826_122503\baseline_violence_c93b12ea-9877-4d9c-bf58-815f2c44e7a6.json".

✅ Completed task 1/4 (25.0%) - baseline/violence in 23.6s
   Est. remaining: 1.6 minutes
Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Basic-Callback-Scan_20250826_122503\baseline_hate_unfairness_19eab094-ae64-40cd-89db-bff5153850c2.json".

✅ Completed task 2/4 (50.0%) - baseline/hate_unfairness in 23.6s
   Est. remaining: 0.5 minutes
Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Basic-Callback-Scan_20250826_122503\flip_violence_82541599-a5f8-4a35-87a7-059131815aab.json".

✅ Completed task 3/4 (75.0%) - flip/violence in 23.6s
   Est. remaining: 0.2 minutes


Scanning: 100%|██████████████████████████████████| 4/4 [00:34<00:00,  8.58s/scan, current=batch 1/1]
Class RedTeamResult: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Basic-Callback-Scan_20250826_122503\flip_hate_unfairness_790f547e-c61b-4de0-baee-58e74c51a2ac.json".

✅ Completed task 4/4 (100.0%) - flip/hate_unfairness in 29.4s
   Est. remaining: 0.0 minutes
Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\red_team_output.json".

Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Basic-Callback-Scan_20250826_122503\final_results.json".

Overall ASR: 0.0%
Attack Success: 0/4 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category     | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
------------------------------------------------------------------

## Advanced Example: Using an Azure Open AI Model Endpoint in a Callback Function

Using the same Azure Open AI model configuration as above, we now wrap it in a callback function for more flexibility and control on the input and output handling. This will demonstrate how to evaluate an actual AI application. To test your own actual AI application, replace the inside of the callback function with a call to your application.

In [8]:
# Define a callback that uses Azure OpenAI API to generate responses
async def azure_openai_callback(
    messages: list,
    stream: Optional[bool] = False,  # noqa: ARG001
    session_state: Optional[str] = None,  # noqa: ARG001
    context: Optional[Dict[str, Any]] = None,  # noqa: ARG001
) -> dict[str, list[dict[str, str]]]:
   

    # Initialize Azure OpenAI client
    client = AzureOpenAI(
        azure_endpoint=azure_openai_endpoint,
        api_version=azure_openai_api_version,
        api_key=azure_openai_api_key
    )

    ## Extract the latest message from the conversation history
    messages_list = [{"role": message.role, "content": message.content} for message in messages]
    latest_message = messages_list[-1]["content"]

    try:
        # Call the model
        response = client.chat.completions.create(
            model=azure_openai_deployment,
            messages=[
                {"role": "user", "content": latest_message},
            ],
            # max_tokens=500, # If using an o1 base model, comment this line out
            max_completion_tokens=500,  # If using an o1 base model, uncomment this line
            # temperature=0.7, # If using an o1 base model, comment this line out (temperature param not supported for o1 base models)
        )

        # Format the response to follow the expected chat protocol format
        formatted_response = {"content": response.choices[0].message.content, "role": "assistant"}
    except Exception as e:
        print(f"Error calling Azure OpenAI: {e!s}")
        formatted_response = "I encountered an error and couldn't process your request."
    return {"messages": [formatted_response]}

In [9]:
# Create the RedTeam instance with all of the risk categories with 5 attack objectives generated for each category
model_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm],
    num_objectives=1,
)

We will use this instance of `model_red_team` to test different attack strategies in the following section.

### Testing Different Attack Strategies

Now we'll run a more comprehensive evaluation using multiple attack strategies across risk categories. This will give us a better understanding of our model's vulnerabilities.

In [ ]:
# Run the red team scan with multiple attack strategies
advanced_result = await model_red_team.scan(
    target=azure_openai_callback,
    scan_name="Advanced-Callback-Scan-Difficult",
    attack_strategies=[
        AttackStrategy.DIFFICULT,  # Group of easy complexity attacks
        # AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        # AttackStrategy.CharacterSpace,  # Add character spaces
        # AttackStrategy.ROT13,  # Use ROT13 encoding
        # AttackStrategy.UnicodeConfusable,  # Use confusable Unicode characters
        # AttackStrategy.CharSwap,  # Swap characters in prompts
        # AttackStrategy.Morse,  # Encode prompts in Morse code
        # AttackStrategy.Leetspeak,  # Use Leetspeak
        # AttackStrategy.Url,  # Use URLs in prompts
        # AttackStrategy.Binary,  # Encode prompts in binary
        #AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),  # Use two strategies in one attack
    ],
    output_path="Advanced-Callback-Scan-Difficult.json",
)

🚀 STARTING RED TEAM SCAN: Advanced-Callback-Scan
📂 Output directory: .\.scan_Advanced-Callback-Scan_20250826_122803
📊 Risk categories: ['violence', 'hate_unfairness', 'sexual', 'self_harm']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/resource/build/redteaming/7ca0694e-23e7-4acb-9ad9-f9f6a56e2d51?wsid=/subscriptions/3c941de1-cd33-4339-bf35-4dafb9f0aff6/resourceGroups/test-agent/providers/Microsoft.CognitiveServices/accounts/agents-build2026-resource/projects/agents-build2026&tid=f79243ce-e61d-4d24-b491-0532f1cc8262
📋 Planning 16 total tasks


Scanning:   0%|                        | 0/16 [00:00<?, ?scan/s, current=fetching baseline/violence]

📚 Using attack objectives from Azure RAI service


Scanning:   0%|                 | 0/16 [00:03<?, ?scan/s, current=fetching baseline/hate_unfairness]

📝 Fetched baseline objectives for violence: 1 objectives


...

Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Advanced-Callback-Scan_20250826_122803\flip_sexual_2e26925e-d486-42e4-817b-afdcfda69a62.json".

✅ Completed task 11/16 (68.8%) - flip/sexual in 58.4s
   Est. remaining: 1.5 minutes
Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Advanced-Callback-Scan_20250826_122803\flip_self_harm_d9a5b87e-e37a-420c-bd06-c5cd742ea7a0.json".

✅ Completed task 12/16 (75.0%) - flip/self_harm in 58.4s
   Est. remaining: 1.1 minutes
Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Advanced-Callback-Scan_20250826_122803\morse_violence_db2abfe5-65eb-4381-8d22-168b0a22b512.json".

✅ Completed task 13/16 (81.2%) - morse/violence in 58.4s
   Est. remaining: 0.8 minutes
Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_A

Scanning:  94%|██████████████████████████████  | 15/16 [03:20<00:09,  9.66s/scan, current=batch 4/4]

Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Advanced-Callback-Scan_20250826_122803\morse_sexual_a47e048d-72b6-4677-8735-78c985c25a6e.json".

✅ Completed task 15/16 (93.8%) - morse/sexual in 59.7s
   Est. remaining: 0.2 minutes
▶️ Starting task: morse strategy for self_harm risk category


Scanning: 100%|████████████████████████████████| 16/16 [03:31<00:00, 13.20s/scan, current=batch 4/4]


Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Advanced-Callback-Scan_20250826_122803\morse_self_harm_497e5df3-305a-4af7-a79c-dcef8dc355ed.json".

✅ Completed task 16/16 (100.0%) - morse/self_harm in 10.2s
   Est. remaining: 0.0 minutes
Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\Advanced-Callback-Scan.json".

Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Advanced-Callback-Scan_20250826_122803\final_results.json".

Overall ASR: 0.0%
Attack Success: 0/16 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category     | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
------------------------------------------------------------

The data and results used in this attack will be saved to the `output_path` specified. The URL printed out at the end of the scorecard will provide a link to where you results are uploaded and logged to your Azure AI Foundry project.

## Bring your own objectives: Using your own prompts as objectives for RedTeam

Below we demonstrate how to use your own prompts as objectives for a `RedTeam` scan. You can see the required format for prompts under `.\data\prompts.json`. Note that when bringing your own prompts, the supported `risk-type`s are `violence`, `sexual`, `hate_unfairness`, and `self_harm`. The number of prompts you specify will be the `num_objectives` used in the scan. 

In [11]:
path_to_prompts = "data\prompts.json"

# Create the RedTeam specifying the custom attack seed prompts to use as objectives
custom_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    custom_attack_seed_prompts=path_to_prompts,  # Path to a file containing custom attack seed prompts
)

In [11]:
custom_red_team_result = await custom_red_team.scan(
    target=azure_openai_callback,
    scan_name="Custom-Prompt-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.DIFFICULT,  # Group of difficult complexity attacks
    ],
    output_path="Custom-Prompt-Scan.json",
)

🚀 STARTING RED TEAM SCAN: Custom-Prompt-Scan
📂 Output directory: .\.scan_Custom-Prompt-Scan_20250815_075153
📊 Risk categories: ['hate_unfairness', 'violence', 'sexual', 'self_harm']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/resource/build/redteaming/75feb9fb-4e22-4104-a17d-b54721eb8ef3?wsid=/subscriptions/3c941de1-cd33-4339-bf35-4dafb9f0aff6/resourceGroups/test-build2025/providers/Microsoft.CognitiveServices/accounts/agents-build2025-resource/projects/agents-build2025&tid=f79243ce-e61d-4d24-b491-0532f1cc8262
📋 Planning 24 total tasks


Scanning:   0%|                                         | 0/24 [00:00<?, ?scan/s, current=batch 1/5]

📚 Using custom attack objectives from data\prompts.json
📝 Fetched baseline objectives for hate_unfairness: 1 objectives
📝 Fetched baseline objectives for violence: 1 objectives
📝 Fetched baseline objectives for sexual: 1 objectives
📝 Fetched baseline objectives for self_harm: 1 objectives
🔄 Fetching objectives for strategy 2/6: base64
🔄 Fetching objectives for strategy 3/6: flip
🔄 Fetching objectives for strategy 4/6: morse
🔄 Fetching objectives for strategy 5/6: tense
🔄 Fetching objectives for strategy 6/6: tense_base64
⚙️ Processing 24 tasks in parallel (max 5 at a time)
▶️ Starting task: baseline strategy for hate_unfairness risk category
▶️ Starting task: baseline strategy for violence risk category
▶️ Starting task: baseline strategy for sexual risk category
▶️ Starting task: baseline strategy for self_harm risk category
▶️ Starting task: base64 strategy for hate_unfairness risk category


Scanning:  17%|█████▌                           | 4/24 [00:40<13:30, 40.55s/scan, current=batch 1/5]

Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Custom-Prompt-Scan_20250815_075153\baseline_hate_unfairness_c552b8f3-4a4e-4dcd-9e05-0681a1e6cf12.json".

✅ Completed task 1/24 (4.2%) - baseline/hate_unfairness in 40.4s
   Est. remaining: 16.7 minutes
Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Custom-Prompt-Scan_20250815_075153\baseline_violence_01467a9b-918c-463a-8f16-b0414c044a26.json".

✅ Completed task 2/24 (8.3%) - baseline/violence in 40.4s
   Est. remaining: 8.0 minutes
Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Custom-Prompt-Scan_20250815_075153\baseline_sexual_65930e78-6c79-4a66-bff1-78c345660bf7.json".

✅ Completed task 3/24 (12.5%) - baseline/sexual in 40.4s
   Est. remaining: 5.1 minutes
Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_tea

...

Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Custom-Prompt-Scan_20250815_075153\tense_self_harm_e44551e4-7449-45ae-9522-cce1745af7ec.json".

✅ Completed task 20/24 (83.3%) - tense/self_harm in 51.9s
   Est. remaining: 0.6 minutes
▶️ Starting task: tense_base64 strategy for hate_unfairness risk category
▶️ Starting task: tense_base64 strategy for violence risk category
▶️ Starting task: tense_base64 strategy for sexual risk category
▶️ Starting task: tense_base64 strategy for self_harm risk category


Scanning:  96%|██████████████████████████████▋ | 23/24 [03:41<00:10, 10.82s/scan, current=batch 5/5]

Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Custom-Prompt-Scan_20250815_075153\tense_base64_hate_unfairness_daed7bb8-17dd-458c-811b-4bdb20c38c0c.json".

✅ Completed task 21/24 (87.5%) - tense_base64/hate_unfairness in 30.1s
   Est. remaining: 0.5 minutes
Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Custom-Prompt-Scan_20250815_075153\tense_base64_violence_07569ea9-e2b1-447f-921e-c1c809777b3e.json".

✅ Completed task 22/24 (91.7%) - tense_base64/violence in 30.1s
   Est. remaining: 0.3 minutes
Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Custom-Prompt-Scan_20250815_075153\tense_base64_sexual_5595d726-3674-4c82-9024-69cd03544fdc.json".

✅ Completed task 23/24 (95.8%) - tense_base64/sexual in 30.1s
   Est. remaining: 0.2 minutes


Scanning: 100%|████████████████████████████████| 24/24 [03:46<00:00,  9.45s/scan, current=batch 5/5]


Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Custom-Prompt-Scan_20250815_075153\tense_base64_self_harm_972b3730-d606-445f-9928-4153ca124fc7.json".

✅ Completed task 24/24 (100.0%) - tense_base64/self_harm in 35.8s
   Est. remaining: 0.0 minutes
Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\Custom-Prompt-Scan.json".

Evaluation results saved to "C:\Users\hifazhassan\workspace\agents\agents-explore-2025\red_teaming_agent\.scan_Custom-Prompt-Scan_20250815_075153\final_results.json".

Overall ASR: 0.0%
Attack Success: 0/24 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category     | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
----------------------------------------------------------

## Conclusion

In this notebook, we've demonstrated how to use the Azure AI Evaluation SDK's `RedTeam` functionality to assess the safety and resilience of AI systems. We started with a basic fixed-response example and then moved to a more realistic model testing across multiple risk categories and attack strategies.

The automated AI red teaming scans provides valuable insights into:

1. **Overall Attack Success Rate (ASR)** - The percentage of attacks that successfully elicit harmful content
2. **Vulnerability by Risk Category** - Which types of harmful content your model is most vulnerable to
3. **Effectiveness of Attack Strategies** - Which attack techniques are most successful against your model
4. **Impact of Complexity** - How more sophisticated attacks affect your model's safety guardrails

By regularly red-teaming your AI applications, you can identify and address potential vulnerabilities before deploying your models to production environments.

### Next Steps

1. **Mitigation**: Use these results to strengthen your model's guardrails against identified attack vectors
2. **Continuous Testing**: Implement regular red team evaluations as part of your development lifecycle
3. **Custom Strategies**: Develop custom attack strategies for your specific use cases and domain
4. **Safety Layers**: Consider adding additional safety layers like Azure AI Content Safety to filter harmful requests and responses 